In [1]:
using GenericTensorNetworks, Graphs
include("utils.jl")

regular3 (generic function with 1 method)

## Hard square entropy constant

In [2]:
F(L) = solve(GenericTensorNetwork(IndependentSet(square(L))), PartitionFunction(0.0))[]^(1/L^2)

F(10)

1.524316200365006

## Configuration enumeration and sampling

In [3]:
solve(GenericTensorNetwork(IndependentSet(petersen)), GraphPolynomial(; method=:finitefield))

0-dimensional Array{Polynomial{BigInt, :x}, 0}:
Polynomial(1 + 10*x + 30*x^2 + 30*x^3 + 5*x^4)

In [4]:
samples = generate_samples(sum(solve(
    GenericTensorNetwork(IndependentSet(ksg(20))),
    ConfigsMax(2; tree_storage=true)
)[].coeffs), 10000);

hamming = hamming_distribution(samples, samples)

321-element Vector{Int64}:
 10000
     0
     0
     0
     0
     0
     0
     0
     0
     0
     ⋮
     0
     0
     0
     0
     0
     0
     0
     0
     0

## Reduce factoring to Rydberg atoms for solution space analysis

In [5]:
using ProblemReductions, UnitDiskMapping, GenericTensorNetworks

factoring_problem = Factoring(4, 5, 221);

result = reduceto(IndependentSet{ProblemReductions.GridGraph{2}, Int, Vector{Int}}, factoring_problem);

mapped_problem = target_problem(result)

IndependentSet{ProblemReductions.GridGraph{2}, Int64, Vector{Int64}}(ProblemReductions.GridGraph{2}([(14, 1), (16, 1), (32, 1), (34, 1), (50, 1), (52, 1), (18, 2), (36, 2), (54, 2), (6, 3)  …  (49, 56), (50, 56), (60, 56), (62, 56), (67, 56), (68, 56), (6, 58), (24, 58), (42, 58), (60, 58)], 2.8567113959936523), [2, 2, 2, 2, 2, 2, 2, 2, 2, 1  …  3, 3, 3, 3, 3, 3, 1, 1, 1, 1])

In [6]:
problem_size(mapped_problem)

(num_vertices = 740, num_edges = 1559)

In [7]:
config = findbest(mapped_problem, GTNSolver());
result = ProblemReductions.read_solution(factoring_problem, extract_solution(result, first(config)))

(13, 17)

## Ground state degeneracy of the Buckyball structure

In [8]:
using GenericTensorNetworks, Graphs, ProblemReductions

function fullerene()  # construct the fullerene graph in 3D space
    th = (1+sqrt(5))/2
    res = NTuple{3,Float64}[]
    for (x, y, z) in ((0.0, 1.0, 3th), (1.0, 2 + th, 2th), (th, 2.0, 2th + 1.0))
        for (a, b, c) in ((x,y,z), (y,z,x), (z,x,y))
            for loc in ((a,b,c), (a,b,-c), (a,-b,c), (a,-b,-c), (-a,b,c), (-a,b,-c), (-a,-b,c), (-a,-b,-c))
                if loc ∉ res
                    push!(res, loc)
                end
            end
        end
    end
    return res
end

fullerene_graph = UnitDiskGraph(fullerene(), sqrt(5)); # construct the unit disk graph
spin_glass = SpinGlass(fullerene_graph, UnitWeight(ne(fullerene_graph)), zeros(Int, nv(fullerene_graph)));
problem_size(spin_glass)

(num_vertices = 60, num_edges = 90)

In [9]:
log(solve(spin_glass, PartitionFunction(1.0))[])/nv(fullerene_graph)

1.3073684577607942

In [10]:
solve(spin_glass, CountingMin())[]

(-66.0, 16000.0)ₜ